In [1]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

**1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

In [2]:
def calc_logloss(y, y_pred):
    for i in y_pred:                                #  ----------- добавила -----------
        if i == 0:                                  #  ----------- добавила -----------
            err = 0.0                               #  ----------- добавила -----------
        else:                                       #  ----------- добавила -----------
            err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err


In [3]:
# Пример применения без 0
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [4]:
# Плохой пример применения с 0
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.0

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [5]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [6]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [7]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [8]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [9]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [10]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    print('W: ', W)
    print('n: ', n)
    print('np.dot(X, W): ', np.dot(X, W))
    print('sigmoid(z): ', sigmoid(np.dot(X, W)))
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
#         print (y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    return W

In [11]:
W = eval_model(X_st, y, iterations=32000, eta=1)

W:  [ 0.49671415 -0.1382643   0.64768854  1.52302986]
n:  10
np.dot(X, W):  [1.24701069 1.51415554 2.96570731 0.82752591 3.76244998 1.78599227
 5.07573744 5.05227806 1.18022448 3.29963837]
sigmoid(z):  [0.77678197 0.81967624 0.95100063 0.69583154 0.97730047 0.85643521
 0.9937923  0.99364588 0.76498816 0.9644164 ]
0 [ 0.11732727 -1.58914029  0.62030812  0.95688359] 1.1785958344356262
3200 [-19.69971896  -2.15981759  -5.07831815  16.3900246 ] 0.16684375125461876
6400 [-27.56595072  -2.86929655  -7.33031741  22.77910414] 0.13241151800390857
9600 [-33.02322626  -3.36225738  -8.87276153  27.19496579] 0.11607491698326718
12800 [-37.30256711  -3.74466013 -10.07949207  30.64164558] 0.10610342849980706
16000 [-40.89301056  -4.06021592 -11.09408531  33.51879367] 0.09912113511864047
19200 [-44.03495993  -4.33081077 -11.98576537  36.02301121] 0.09379851399459645
22400 [-46.863349    -4.56888384 -12.79286435  38.26474586] 0.08950339523244644
25600 [-49.46104065  -4.78217478 -13.53865245  40.3119518

*минимальная ошибка составила 0,082 при параметрах шага = 1 и кол-во итераций 32000. Если брать шаг 1,2, то это уже будет большой шаг, т.к. ошибка начинает то увеличиваться, то уменьшаться, поэтому шаг взять как 1. А если брать больше итераций, то ошибка выводится как nan.*

**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [12]:
def calc_pred_proba(X, W):

    y_pred_proba = 1 / (1 + np.exp(- (np.dot(X, W))))
    return y_pred_proba

In [13]:
calc_pred_proba(X_st, W)

array([3.24586616e-01, 1.02810703e-03, 1.00000000e+00, 2.48070153e-11,
       9.77082842e-01, 8.17372181e-03, 1.00000000e+00, 5.28863441e-03,
       6.90783806e-01, 1.00000000e+00])

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [14]:
def calc_pred(W, X):
    m = X.shape[0]
    
    y_pred = np.zeros(m)

    
    A = np.squeeze(sigmoid(np.dot(X, W)))

    # За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(A.shape[0]):
        if (A[i] > 0.5): 
            y_pred[i] = 1
        elif (A[i] <= 0.5):
            y_pred[i] = 0

    return y_pred

In [15]:
calc_pred(W, X_st)

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

**6. Могла ли модель переобучиться? Почему?**

*Считаю, что модель не переобучилась, т.к. ошибка скатывается к минимуму плавно без разбросов в + или - от текущего значения ошибки.*